In [119]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
from torch import nn

### read data

In [120]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')

In [121]:
train.head()

,row_id,productyear,productquarter,productdayofweek,productdayofmonth,productweekday,productis_weekend,store,num_sold
0,0,2014,0,2,30,2,-1,0,329
1,1,2015,1,3,31,3,0,0,520
2,2,2013,-1,1,29,1,-2,0,146
3,3,2014,0,2,30,2,-1,1,572
4,4,2015,1,3,31,3,0,1,911


### data preprocessing

In [122]:
train = train.set_index('row_id')
test = test.set_index('row_id')


In [123]:
train

,productyear,productquarter,productdayofweek,productdayofmonth,productweekday,productis_weekend,store,num_sold
row_id,,,,,,,,
0,2014,0,2,30,2,-1,0,329
1,2015,1,3,31,3,0,0,520
2,2013,-1,1,29,1,-2,0,146
3,2014,0,2,30,2,-1,1,572
4,2015,1,3,31,3,0,1,911
...,...,...,...,...,...,...,...,...
26293,2018,4,0,31,0,0,0,823
26294,2016,2,-2,29,-2,-2,0,250
26295,2017,3,-1,30,-1,-1,1,1004


In [124]:
x_data = train.drop('num_sold', axis=1)
y_data = train.num_sold

In [125]:
x_data = torch.tensor(x_data.values, dtype=torch.float32)
test = torch.tensor(test.values, dtype=torch.float32)

y_data = torch.tensor(y_data.values, dtype=torch.float32)

In [126]:
print(x_data[:10])

print(x_data.shape)

tensor([[ 2.0140e+03,  0.0000e+00,  2.0000e+00,  3.0000e+01,  2.0000e+00,
         -1.0000e+00,  0.0000e+00],
        [ 2.0150e+03,  1.0000e+00,  3.0000e+00,  3.1000e+01,  3.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.0130e+03, -1.0000e+00,  1.0000e+00,  2.9000e+01,  1.0000e+00,
         -2.0000e+00,  0.0000e+00],
        [ 2.0140e+03,  0.0000e+00,  2.0000e+00,  3.0000e+01,  2.0000e+00,
         -1.0000e+00,  1.0000e+00],
        [ 2.0150e+03,  1.0000e+00,  3.0000e+00,  3.1000e+01,  3.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 2.0130e+03, -1.0000e+00,  1.0000e+00,  2.9000e+01,  1.0000e+00,
         -2.0000e+00,  1.0000e+00],
        [ 2.0140e+03,  0.0000e+00,  2.0000e+00,  3.0000e+01,  2.0000e+00,
         -1.0000e+00,  0.0000e+00],
        [ 2.0150e+03,  1.0000e+00,  3.0000e+00,  3.1000e+01,  3.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.0130e+03, -1.0000e+00,  1.0000e+00,  2.9000e+01,  1.0000e+00,
         -2.0000e+00,  0.0000e+00],
        [ 

### model train

In [127]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

In [128]:
from torch.utils.data import TensorDataset

In [129]:
from torch.utils.data import DataLoader

In [130]:
dataset_train = TensorDataset(x_train, y_train)
loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
loader_test = DataLoader(dataset_test, batch_size=256, shuffle=True)


In [131]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [137]:
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(7, 14)
        self.bn1 = nn.BatchNorm1d(14)
        self.fc2 = nn.Linear(14, 28)
        self.bn2 = nn.BatchNorm1d(28)
        # self.fc3 = nn.Linear(28, 56)
        # self.bn3 = nn.BatchNorm1d(56)
        # self.fc4 = nn.Linear(56, 28)
        # self.bn4 = nn.BatchNorm1d(28)
        self.fc5 = nn.Linear(28, 14)
        self.bn5 = nn.BatchNorm1d(14)
        self.fc6 = nn.Linear(14, 7)
        self.fc7 = nn.Linear(7, 1)



    def forward(self, x):
        x = self.fc1(x)
        x = F.tanh(self.bn1(x))
        x = F.relu(self.fc2(x))
        # x = F.tanh(self.bn2(x))
        # x = F.relu(self.fc3(x))
        # x = F.tanh(self.bn3(x))
        # x = F.relu(self.fc4(x))
        # x = F.tanh(self.bn4(x))
        x = F.relu(self.fc5(x))
        x = F.tanh(self.bn5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)


        
        return x

In [138]:
model = NeuralNetwork().to(device)
# model.apply(init_weights)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=7, out_features=14, bias=True)
  (bn1): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=14, out_features=28, bias=True)
  (bn2): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=28, out_features=14, bias=True)
  (bn5): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=14, out_features=7, bias=True)
  (fc7): Linear(in_features=7, out_features=1, bias=True)
)


In [139]:
learning_rate = 1e-3
batch_size = 512
epochs = 100

In [140]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        
        X = X.to(device)
        y = y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss = loss.item()
    loss = loss ** 0.5
    print(f"loss: {loss:>7f}")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    test_loss = test_loss ** 0.5
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [141]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(loader_train, model, loss_fn, optimizer)
    test_loop(loader_test, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


/home/activus/.local/lib/python3.8/site-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/activus/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/activus/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/activus/.local/lib/python3.8/site-packages/torch/nn/module

loss: 560.667236
Test Error: 
 Avg loss: 426.084505 

Epoch 2
-------------------------------
loss: 441.197306
Test Error: 
 Avg loss: 388.978419 

Epoch 3
-------------------------------
loss: 395.070286
Test Error: 
 Avg loss: 359.554485 

Epoch 4
-------------------------------
loss: 315.032154
Test Error: 
 Avg loss: 336.552690 

Epoch 5
-------------------------------
loss: 296.728727
Test Error: 
 Avg loss: 318.936631 

Epoch 6
-------------------------------
loss: 341.233882
Test Error: 
 Avg loss: 304.756930 

Epoch 7
-------------------------------
loss: 338.831344
Test Error: 
 Avg loss: 294.636905 

Epoch 8
-------------------------------
loss: 452.021588
Test Error: 
 Avg loss: 286.120807 

Epoch 9
-------------------------------
loss: 160.735201
Test Error: 
 Avg loss: 280.132750 

Epoch 10
-------------------------------
loss: 176.087652
Test Error: 
 Avg loss: 275.820313 

Epoch 11
-------------------------------
loss: 171.779405
Test Error: 
 Avg loss: 272.621773 

Epoc

In [142]:
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)


In [143]:
x_train = x_train.to(device)
x_test = x_test.to(device)

train_pred = model(x_train)
test_pred = model(x_test)

train_score = SMAPE(y_train.cpu().detach().numpy(), train_pred.cpu().detach().numpy())
test_score = SMAPE(y_test.cpu().detach().numpy(), test_pred.cpu().detach().numpy())

print('train_score ', train_score, 'test_score', test_score)

train_score  52.401234 test_score 53.169334


In [ ]:
test = test.to(device)

In [ ]:
test_pred = model(test)

In [ ]:
test_pred

tensor([[389.6154],
        [389.6154],
        [389.6154],
        ...,
        [389.6154],
        [389.6154],
        [389.6154]], grad_fn=<AddmmBackward0>)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission.num_sold = test_pred.cpu().detach().numpy().reshape(-1)
submission.to_csv('submission.csv', index=False)